``
pip install pandas numpy scikit-learn six matplotlib gmdhpy
``

In [1]:
import pandas as pd

from gmdhpy.gmdh import MultilayerGMDH

In [2]:
# 1. 加载数据 txt格式,\t为分隔符
clinical_data = pd.read_csv('Training_clinical_infor.txt', sep='\t')
test_clinical_data = pd.read_csv('testing_clinical_infor.txt', sep='\t')

gene_expression_data = pd.read_csv('Training_selected_genes.txt', sep='\t')
gene_expression_data.set_index(gene_expression_data.columns[0], inplace=True)  # 将第一列（基因名）设为索引
test_gene_expression_data = pd.read_csv('testing_selected_genes.txt', sep='\t')
test_gene_expression_data.set_index(test_gene_expression_data.columns[0], inplace=True)  # 将第一列（基因名）设为索引

# clinical id 替换 - 为 .
clinical_data['id'] = clinical_data['id'].str.replace('-', '.')

# print(clinical_data.head(5))
print(gene_expression_data.head(5))
print(test_gene_expression_data.head(5))


            TCGA.P5.A781.01A.11R.A32Q.07  TCGA.S9.A6WL.01A.21R.A33Z.07  \
Unnamed: 0                                                               
RPAP1                        2284.529555                   2498.443645   
RNF130                       7217.447970                   5749.241712   
S1PR3                         333.990256                    967.611382   
IL18                          237.142133                    173.459492   
DCTD                          844.479428                   1430.518339   

            TCGA.P5.A737.01A.11R.A32Q.07  TCGA.S9.A6U5.01A.12R.A33Z.07  \
Unnamed: 0                                                               
RPAP1                        2929.385527                   3210.650243   
RNF130                       7306.794324                   6120.424533   
S1PR3                         221.148604                    965.351187   
IL18                          147.802836                    496.886347   
DCTD                          680.115

In [7]:
# 2. 处理test、train数据不同

# 统一两个数据集的基因表达数据
def unify_row(tb1,tb2):
    print(f"保留共同基因……")
    # 找到两个表中共同的基因
    common_genes = tb1.index.intersection(tb2.index)

    # 打印信息
    print(f"tb1基因数: {len(tb1.index)} , tb2基因数: {len(tb2.index)} , 共同基因数量: {len(common_genes)}")
    print(f"tb1独有基因列表: {set(tb1.index) - set(common_genes)}")
    print(f"tb2独有基因列表: {set(tb2.index) - set(common_genes)}")

    # 只保留共同基因的表达数据
    tb1 = tb1.loc[common_genes]
    tb2 = tb2.loc[common_genes]
    
    return tb1, tb2

# 转置基因表达数据，使列名变为行索引，行名变成列索引，方便后续拼接
def transpose_gene_expression_data(gene_expression_df):
    gene_expression_t = gene_expression_df.T  # 转置：列名变为行索引，行名（基因名）变成列索引
    gene_expression_t.reset_index(inplace=True)  # 将索引变为列
    gene_expression_t.rename(columns={'index': 'id'}, inplace=True)  # 重命名索引列为id

    # 按 数字、字母顺序进行列排序(从2列开始)
    gene_1_columns = gene_expression_t.iloc[:, 0]  # 保留第一列数据（id）

    gene_expression_t = gene_expression_t.reindex(sorted(gene_expression_t.columns[1:]), axis=1)
    gene_expression_t.insert(0, 'id', gene_1_columns)  # 将id列放回第一列

    return gene_expression_t


gene_unified, test_gene_unified = unify_row(gene_expression_data, test_gene_expression_data)
print(gene_unified.shape)
print(test_gene_unified.shape)

gene_expression_transposed = transpose_gene_expression_data(gene_unified)
test_gene_expression_transposed = transpose_gene_expression_data(test_gene_unified)

print(gene_expression_transposed.head())
print(test_gene_expression_transposed.head())


保留共同基因……
tb1基因数: 1611 , tb2基因数: 1587 , 共同基因数量: 1587
tb1独有基因列表: {'AC004540.2', 'AL355974.2', 'LSP1', 'TMIGD3', 'FLJ16779', 'AC004656.1', 'AC090114.2', 'CASTOR3', 'PI4K2A', 'AL139393.2', 'LINC01094', 'AL118505.1', 'AC093010.3', 'AC008124.1', 'BAHCC1', 'PAX8-AS1', 'TAP2', 'ZNF710-AS1', 'AQP1', 'LINC01578', 'SLC5A3', 'LINC02283', 'CSF2RA', 'PCDHB7'}
tb2独有基因列表: set()
(1587, 496)
(1587, 927)
Unnamed: 0                            id          A2M        AAGAB  \
0           TCGA.P5.A781.01A.11R.A32Q.07  23443.58162  1435.524515   
1           TCGA.S9.A6WL.01A.21R.A33Z.07  37824.61859  1942.537321   
2           TCGA.P5.A737.01A.11R.A32Q.07  20450.13371  1063.513638   
3           TCGA.S9.A6U5.01A.12R.A33Z.07  23649.63401  1518.100497   
4           TCGA.HT.7607.01A.11R.2090.07  26284.19134  2835.889111   

Unnamed: 0       ABCA7        ABCC1       ABCC3        ABCC4      ABHD14B  \
0           352.997832   863.487003    7.240981  1901.662677  1257.215354   
1           706.377208   877.746826 

In [4]:
# 3. 合并数据，指定x和y
train_data = pd.merge(clinical_data, gene_expression_transposed, left_on='id', right_on='id')
print(train_data.head())

test_data = pd.merge(test_clinical_data, test_gene_expression_transposed, left_on='id', right_on='id')
print(test_data.head())


                             id  OS.time  OS.state Transcriptome.Subtype  \
0  TCGA.P5.A781.01A.11R.A32Q.07    134.0       0.0                    NE   
1  TCGA.S9.A6WL.01A.21R.A33Z.07    946.0       0.0                    NE   
2  TCGA.P5.A737.01A.11R.A32Q.07    372.0       0.0                    PN   
3  TCGA.S9.A6U5.01A.12R.A33Z.07    992.0       0.0                    PN   
4  TCGA.HT.7607.01A.11R.2090.07     96.0       1.0                    NE   

  diagnoses.primary_diagnosis Chr.1p_19q.codeletion Transcriptome.Subtype.1  \
0     Astrocytoma, anaplastic                 codel                  Mutant   
1     Astrocytoma, anaplastic                 codel                  Mutant   
2            Astrocytoma, NOS                 codel                  Mutant   
3            Astrocytoma, NOS                 codel                  Mutant   
4            Astrocytoma, NOS                 codel                  Mutant   

  2021classification_created By BZT demographic.vital_status   age  

In [5]:
# 4. 数据预处理

# 清除 OS.time < 10 的行
train_data = train_data[train_data['OS.time'] >= 10]
test_data = test_data[test_data['OS.time'] >= 10]

# 清除 OS.status == 0 且 OS.time < 100 的行
train_data = train_data[~((train_data['OS.state'] == 0) & (train_data['OS.time'] < 100))]
test_data = test_data[~((test_data['OS.state'] == 0) & (test_data['OS.time'] < 100))]


In [6]:
# 5. 划分训练集和测试集
# id	OS.time	OS.state	Transcriptome.Subtype	diagnoses.primary_diagnosis	Chr.1p_19q.codeletion	Transcriptome.Subtype.1	2021classification_created By BZT	demographic.vital_status	age	demographic.gender	Grade
X_train = train_data.drop(['id', 'OS.time','OS.state','Transcriptome.Subtype','diagnoses.primary_diagnosis','Chr.1p_19q.codeletion','Transcriptome.Subtype.1','2021classification_created By BZT','demographic.vital_status','age','demographic.gender','Grade'], axis=1)
Y_train = train_data[['OS.time']]  # 目标变量：生存时间和生存状态

# id	OS.time	OS.state	subtype	IDH stat	PRS_type	Histology	Grade	Gender	Radio_status (treated=1;un-treated=0)	Chemo_status (TMZ treated=1;un-treated=0)	1p19q_codeletion_status	2021classification_created By BZT
X_test = test_data.drop(['id', 'OS.time','OS.state','subtype','IDH stat','PRS_type','Histology','Grade','Gender','Radio_status (treated=1;un-treated=0)','Chemo_status (TMZ treated=1;un-treated=0)','1p19q_codeletion_status','2021classification_created By BZT'], axis=1)
Y_test = test_data[['OS.time']]  # 目标变量：生存时间和生存状态


print(X_train.head())
print(Y_train.head())
print(X_test.head())
print(Y_test.head())

# 转成numpy
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()

           A2M        AAGAB       ABCA7        ABCC1       ABCC3        ABCC4  \
0  23443.58162  1435.524515  352.997832   863.487003    7.240981  1901.662677   
1  37824.61859  1942.537321  706.377208   877.746826   81.505062  1101.363280   
2  20450.13371  1063.513638  115.575150  1051.289343   14.446894   696.784797   
3  23649.63401  1518.100497  605.672116  1271.127401  315.576733  1312.289584   
4  26284.19134  2835.889111  440.458586   970.057600  505.129093  1172.808378   

       ABHD14B        ABI3   AC004540.2   AC004656.1  ...       ZNF395  \
0  1257.215354  433.553747  1089.767664   630.870483  ...  1766.799403   
1  1706.381628  206.897466   621.737335  1827.594285  ...  1201.677203   
2   996.835667  385.620932    10.001696   383.398333  ...  2469.307527   
3  1745.472553  859.505575  1136.860282   522.367698  ...  1802.315567   
4  1075.802618  444.828215    27.091699  3046.505221  ...  1764.456122   

        ZNF397       ZNF436       ZNF512      ZNF512B       ZNF652  

In [7]:
print(X_train.shape)
print(Y_train.shape)

print(X_train)

(447, 1611)
(447, 1)
[[23443.58162    1435.524515    352.9978315 ...  4845.121518
   1340.486637   3521.832211 ]
 [37824.61859    1942.537321    706.3772078 ...  1364.687328
   1589.348718   6955.098661 ]
 [20450.13371    1063.513638    115.5751498 ...  2747.132406
   2598.218271   6048.803271 ]
 ...
 [83518.62893    2967.619397    240.7868215 ...  1613.584414
    892.7875004 20110.39024  ]
 [39158.49854    1828.496882    437.2177817 ...  1152.401835
   1908.122637  11648.52408  ]
 [60943.17279    1766.271049    277.0621253 ...  1497.604745
   1064.170436  27104.86178  ]]


In [8]:
# 6. 训练GMDH模型
model = MultilayerGMDH()  # 初始化GMDH模型
model.fit(X_train, Y_train)  # 训练模型


train layer0 in 770.32 sec
Error training model on train data set, layer index 1


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:

# 7. 模型评估
predictions = model.predict(X_test)